## Lucas Kanade Optical Flow Algoritması

In [ ]:
import cv2
import numpy as np
def draw_tracks_on_frame(frame, p0, p1, st):
    output = frame.copy()
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    for new, old in zip(good_new, good_old):
        a, b = new.ravel()
        c, d = old.ravel()

        vec_x = a - c
        vec_y = b - d

        scale = 7.0
        end_point = (int(c + vec_x * scale), int(d + vec_y * scale))
        start_point = (int(c), int(d))

        output = cv2.arrowedLine(output, start_point, end_point,
                                 color=(0, 0, 255), thickness=1, tipLength=0.02)
        output = cv2.circle(output, end_point, 1, (0, 0, 0), -1)
    return output

def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    input_fps = cap.get(cv2.CAP_PROP_FPS)
    slowdown_factor = 1  
    fps = input_fps / slowdown_factor

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=200, qualityLevel=0.01, minDistance=7, blockSize=7)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None,
                                               winSize=(15, 15), maxLevel=2,
                                               criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

        if p1 is None:
            break

        tracked_frame = draw_tracks_on_frame(frame, p0, p1, st)
        out.write(tracked_frame)

        old_gray = frame_gray.copy()
        p0 = p1[st == 1].reshape(-1, 1, 2)

    cap.release()
    out.release()
video_path = "C:/Users/90551/Desktop/EE584/Term Project/kanade_deneyler/ball_original.mp4"
output_path = "C:/Users/90551/Desktop/EE584/Term Project/kanade_deneyler/flow_output_original.mp4"
process_video(video_path, output_path)


## Video blurlamak için

In [ ]:
import cv2
import os

in_path  = r"C:/Users/90551/Desktop/EE584/Term Project/kanade_deneyler/ball_original.mp4"
sigma    = 37
out_path = r"C:/Users/90551/Desktop/EE584/Term Project/kanade_deneyler/video_blurred_"+str(sigma)+".mp4"
ksize = max(3, int(6 * sigma + 1))
if ksize % 2 == 0:          
    ksize += 1

os.makedirs(os.path.dirname(out_path), exist_ok=True)

cap = cv2.VideoCapture(in_path)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")            
fps    = cap.get(cv2.CAP_PROP_FPS)
w      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        blurred = cv2.GaussianBlur(frame, (ksize, ksize), sigmaX=sigma, sigmaY=sigma)
        writer.write(blurred)
finally:
    cap.release()
    writer.release()

